.NET 7 introduced two new mechanisms to improve management of minimal API 
- `Filters`, help ***separate*** `validation checks` and `cross-cutting functions` such as logging from the important logic in your endpoint handler functions.  
- `Route groups`, help ***reduce duplication*** by `applying filters` and `routing` to multiple handlers at the same time.

In [ ]:
app.MapGet("/fruits", () => {/* */});
app.MapGet("/fruits/{id}", (string id) => {/* */});
app.MapPost("/fruits/{id}", (Fruit fruit, string id) => {/* */});
app.MapPut("/fruits/{id}", (Fruit fruit, string id) => {/* */});
app.MapDelete("/fruits/{id}", (string id) => {/* */});

In [ ]:

using System.Collections.Concurrent;

WebApplicationBuilder builder = WebApplication.CreateBuilder(args);
WebApplication app = builder.Build();

app.AddFruitApi(); 

app.Run();


public static class FruitApiExtensions
{
    public static void AddFruitApi(this WebApplication app)
    {
        var _fruit = new ConcurrentDictionary<string, Fruit>();

        RouteGroupBuilder fruitApi = app.MapGroup("/fruit");
        fruitApi.MapGet("/", () => _fruit);

        RouteGroupBuilder fruitApiWithValidation = fruitApi.MapGroup("/")
            .AddEndpointFilter<IdValidationFilter>();

        fruitApiWithValidation.MapGet("/{id}", (string id) =>
            _fruit.TryGetValue(id, out var fruit)
            ? TypedResults.Ok(fruit)
            : Results.Problem(statusCode: 404));

        fruitApiWithValidation.MapPost("/{id}", (Fruit fruit, string id) =>
            _fruit.TryAdd(id, fruit)
            ? TypedResults.Created($"/fruit/{id}", fruit)
            : Results.ValidationProblem(new Dictionary<string, string[]>
            {
                { "id", new[] { "A fruit with this id already exists" } }
            }));

        fruitApiWithValidation.MapPut("/{id}", (string id, Fruit fruit) =>
        {
            _fruit[id] = fruit;
            return Results.NoContent();
        });

        fruitApiWithValidation.MapDelete("/{id}", (string id) =>
        {
            _fruit.TryRemove(id, out _);
            return Results.NoContent();
        });
    }
}

public record Fruit(string Name, int Stock);

public class IdValidationFilter : IEndpointFilter
{
    public async ValueTask<object?> InvokeAsync(
        EndpointFilterInvocationContext context,
        EndpointFilterDelegate next)
    {
        var id = context.GetArgument<string>(0);
        if (string.IsNullOrEmpty(id) || !id.StartsWith('f'))
        {
            return Results.ValidationProblem(
            new Dictionary<string, string[]>
            {
                {"id", new[]{"Invalid format. Id must start with 'f'"}}
            });
        }
        return await next(context);
    }
}